Didn't miss writing a notebook for statistics that much :|

First things first we need to change the extensions of the matches to txt from tcx (the export format of choice of the garmin connect app) but first some imports

In [2]:
import os
import numpy as np
import pandas as pd
import csv
import chess
import chess.pgn
import chess.engine
from stockfish import Stockfish


In [3]:
def change_ext(cartella, old_ext, new_ext):
    for root, dirs, files in os.walk(cartella):
        # print(os.getcwd())
        for f in files:
            filename = os.fsdecode(f)
            
                
            if filename.endswith(old_ext):
                name, ext = os.path.splitext(filename)
                
                new_name = name + new_ext
                jpath = os.path.join(root,new_name)
                os.rename(os.path.join(root,filename), jpath)
                print(f"renamed {os.path.join(root,filename)} to {jpath}")

change_ext("games", ".tcx", ".txt")

Now we have to import the data from the [Portable Game Notation or pgn](https://en.wikipedia.org/wiki/Portable_Game_Notation)
and read the heartbeats of both white and black (in the games where they are both available) in order to create a 3d dataframe for every game and then to visualize the data we are working with. To process the portable game notation we will use the following cell of code.

In [65]:
### UTILS FOR READING THE ACTIVITIES

def calctime(start,curr):
    c = [int(x) for x in curr.split(":")]
    s = [int(x) for x in start.split(":")]
    return ((c[0] - s[0]) * 60 + (c[1] - s[1])) * 60 + (c[2] - s[2])

def read_activity(game):
    with open(game) as f:
        lines = f.readlines()
        hr = []
        time = []
        for s in lines:
            if "<Value>" in s:
                hr.append(int(s.strip()[s.strip().find(">") + 1:s.strip().rfind("<")]))
            if "<Time>" in s:
                s = s.strip()[3:]
                if not time:
                    start = s.strip()[s.strip().find("T") + 1:s.strip().rfind(".")]
                    
                time.append(calctime(start, s.strip()[s.strip().find("T") + 1:s.strip().rfind(".")]))
        
        #print(f"hr is {hr}, and time is {time}")
        hr = hr[2:]
        if len(time) - 1 != time[-1]:
            diz = {time[i]: hr[i] for i in range(min(len(hr), len(time)))}
            last_battito = 0
            hr = []
            tempo = []
            keys = list(diz.keys())
            
            print(time[-1] + 1)
            prev = -1
            last_battito = 0
            for s in range(time[-1] + 1):
                if s in diz:
                    
                    hr.append(diz[s])
                    tempo.append(s)
                    last = (diz[s], s)
                    indice_s = keys.index(s)
                    if indice_s + 1 < len(keys):
                        next_s = keys[indice_s + 1]
                        next = (diz[keys[indice_s + 1]], next_s)
                    
                    
                    
                else:
                    total_gap = s - last[1] + next[1] - s
                    if total_gap <= 0:
                        
                        hr.append(last_battito)
                        tempo.append(s)
                        continue
                    
                    weight_next = (s-last[1]) * 100 / total_gap
                    weight_last = 100 - weight_next 
                    est_battito = (weight_last * float(last[0]) + weight_next * float(next[0])) / 100
                    last_battito = est_battito
                    
                    hr.append(round(est_battito))
                    tempo.append(s)
                    
            delta = sum(hr[:10]) // 10
            hr = np.array(hr) - delta
            
            return list(zip(hr,tempo))
                
        delta = sum(hr[:10]) // 10
        hr = np.array(hr) - delta
        return list(zip(hr,time))
        
#read_activity("games/partita_0814-2/white.txt")

In [50]:
lista = [(1,0),(2,0),(3,0),(4,0),(5,0),(3,0),(2,0),(1,0)]
delta = sum([lista[i][0] for i in range(3)]) // 3


array([[-1, -2],
       [ 0, -2],
       [ 1, -2],
       [ 2, -2],
       [ 3, -2],
       [ 1, -2],
       [ 0, -2],
       [-1, -2]])

In [5]:
### UTILS FOR READING THE PGN

#this method takes as input the board and the last move so to update the board and compute the evaluation
def get_evaluation(board,move):

    stockfish = Stockfish("/usr/local/Cellar/stockfish/15/bin/stockfish")     #if no parameter passed, takes as input the executable file called stockfish from the same directory, if found
    board.push_san(move)        #insert the move in the board
    stockfish.set_fen_position(board.fen())  #bpard.fen() applies a PGN to FEN conversion
    engine = chess.engine.SimpleEngine.popen_uci("stockfish")
    info = engine.analyse(board, chess.engine.Limit(time=0.1))     #gets the infos of the game
    
    #if the position on the board is not a mate the score is in centipawns, then it's divided by 100 to get the actual score
    #Gets the score from White’s point of view
    if chess.engine.PovScore.is_mate(info['score']):
        eval = chess.engine.PovScore.white(info['score'])
    else:
        eval = int(str(chess.engine.PovScore.white(info['score'])))/100

    return board, eval      #returns the board updated and the evaluation of the position

In [6]:
def get_timestamps(file):
    timestamps = []
    file = file[:-3] + "txt"
    with open(file, "r") as f:
        file_read = f.read()
        lista = file_read.split("%")
        
        
        for el in lista:
            if el[0:4] == "time":
                num = ""
                flag = False
                for i  in range(len(el)):
                    while el[i].isdigit():
                        num += el[i]
                        i += 1
                        flag = True
                    if flag:
                        timestamps.append(int(num)/10)
                        break
    return timestamps
            
get_timestamps("/Users/tommasodimario/Documents/GitHub/statistics-101/project/src/games/partita_0816-7/tommidim_vs_roccot01_2022.08.16.txt")

[0.1,
 0.1,
 3.4,
 0.9,
 13.6,
 1.9,
 2.8,
 9.8,
 3.5,
 11.4,
 13.7,
 9.5,
 13.0,
 19.4,
 12.9,
 11.3,
 2.5,
 1.7,
 17.7,
 5.5,
 10.1,
 2.7,
 0.6,
 4.8,
 21.2,
 4.4,
 0.1,
 10.4,
 2.5,
 22.1,
 10.8,
 12.1,
 23.1,
 6.5,
 1.6,
 20.3,
 7.7,
 49.1,
 27.4,
 10.1,
 3.5,
 16.7,
 6.2,
 8.7,
 37.7,
 34.0,
 49.1,
 9.7,
 26.9,
 29.2,
 11.8,
 1.4,
 12.9,
 22.8,
 17.7,
 17.7,
 12.9,
 20.9,
 54.3,
 12.6,
 6.0,
 1.6,
 0.4,
 1.0,
 0.4,
 26.9,
 3.7,
 7.1,
 25.2,
 15.9,
 42.5,
 7.4,
 15.1,
 16.0,
 10.7,
 21.5,
 7.9,
 13.1,
 21.0,
 1.7,
 2.0,
 22.3,
 1.1,
 19.5,
 6.0,
 18.0,
 19.3,
 12.7,
 0.7,
 3.9,
 2.0,
 6.2,
 3.8,
 1.3,
 3.1,
 3.2,
 7.7,
 1.5,
 0.1,
 1.1]

In [95]:
def get_moves(file):
    moves = []
    file = file[:-3] + "txt"
    with open(file, "r") as f:
        file_read = f.read()
        lista = file_read.split("{")
        return [[elem.strip()]  for el in lista for elem in el[-7:].split() if "." not in elem][:-1]
        
get_moves("/Users/tommasodimario/Documents/GitHub/statistics-101/project/src/partite_confronto/partita_timestamp.txt")

[['e4'],
 ['e5'],
 ['Nf3'],
 ['Nc6'],
 ['Nc3'],
 ['Bb4'],
 ['a3'],
 ['Bxc3'],
 ['dxc3'],
 ['Nf6'],
 ['Bd3'],
 ['d5'],
 ['exd5'],
 ['Qxd5'],
 ['O-O'],
 ['e4'],
 ['Re1'],
 ['O-O'],
 ['Bxe4'],
 ['Nxe4'],
 ['Qxd5'],
 ['0]}']]

In [70]:
#read_pgn takes as input the name of the file pgn containing the game and returns a dataframe
def read_pgn(game_name): 
    old = False
    if ("05.29" in game_name or "05.25" in game_name or "clk" in game_name) and "x" not in game_name:
        old = True
    #opens and reads the pgn file
    if not old:
        timestamps = get_timestamps(game_name)
    pgn = open(game_name) 
    
    game = chess.pgn.read_game(pgn)
    
    print(old)
                                   
    array = []      #initialize the array that will contain the data
    index_values = [] 
    i=0
    time_white = -0.1      #to keep track of the time for each player
    time_black = 0      #to keep track of the time for each player
    board = chess.Board()       #initialize the chessboard
    
    #iterate through the moves
    for node in game.mainline():        
        print(node.move)
        move = str(node.move)       #represents the last move
        board,evaluation = get_evaluation(board,move)       #the get_evaluation method returns the updated board and the evaluation of the current position
        
        
        if i%2 == 0:        #if the white is moving we update his time
            index_values.append("white")
            if old:
                time_white = 600 - node.clock()
            else:
                time_white += timestamps[i]
                array.append([str(node.move),round(time_white,4),round(time_white+time_black,4),evaluation])
            
           
        else:           #if the black is moving we update his time
            index_values.append("black") 
            if old:
                time_black = 600 - node.clock()
            else:
                time_black += timestamps[i]
                array.append([str(node.move),round(time_black,4),round(time_white+time_black,4),evaluation])

        #evaluation = get_evaluation(board,move)
        #we append an element in the array (a row in the dataframe) with move, time the player used, total time since the game started and evaluation of the position
        # if str(evaluation)[:2] == "#+":
        #     evaluation = 15
        # elif str(evaluation)[:2] == "#-":
        #     evaluation = -15
        if old:
            array.append([str(node.move),round(600 - node.clock(),4),round(time_white+time_black,4),evaluation])   
        i+=1
    print("\n\n The array is:\n")
    print(array)
    print("\n\n\n\n")
    
    column_values = ['move','time','real time','evaluation'] #create the columns names
    
    res = []
    j = 0
    #df = pd.DataFrame(data = array, index = index_values, columns = column_values) 
    
    tub = 0
    tuw = 0
    pos = 0.0
    for i in range(int(array[-1][2]) + 1):
        mosse = []
        
        while j < len(array):
            if float(array[j][2]) <= float(i):
                break
            
            print(f"array[j] is {array[j]}")
            pos = array[j][3]
            mosse.append(array[j][0])
            if j % 2 == 0:
                tuw = array[j][1]
            elif j % 2 != 0:
                tub = array[j][1]
            j += 1
            if j > len(array):
                break
        res.append([i, pos, (tuw,tub), mosse])
        
    print("\n\n The res is:\n")
    print(res)
    print("\n\n\n\n")
         
        
    #create a dataframe

    return res


read_pgn("partite_confronto/partita_timestamp.pgn")

False
e2e4
e7e5
g1f3
b8c6
b1c3
f8b4
a2a3
b4c3
d2c3
g8f6
f1d3
d7d5
e4d5
d8d5
e1g1
e5e4
f1e1
e8g8


KeyboardInterrupt: 

In [8]:
## creaiamo un dizionario con tutte le partite, il dizionario viene inizializzato qui così poi dopo verrano fatti 
# gli accessi da sotto e poi vederemo il da farsi

diz_pgn = {} 
os.chdir("/Users/tommasodimario/Documents/GitHub/statistics-101/project/src")
os.getcwd()
listozzo = [('zSanjatosti_vs_valesepicacchi_2022.08.14 (1).pgn', 'games/partita_0814-2'), ('roccot01_vs_tommidim_2022.08.16.pgn', 'games/partita_0816-6'), ('tommidim_vs_roccot01_2022.08.16.pgn', 'games/partita_0816-1'), ('tommidim_vs_roccot01_2022.08.16.pgn', 'games/partita_0816-7'), ('tommidim_vs_roccot01_2022.08.16.pgn', 'games/partita_0816-3_pgn_occhio'), ('gggarce_vs_valesepicacchi_2022.08.13 (2).pgn', 'games/partita_0813-3'), ('valesepicacchi_vs_gggarce_2022.08.13.pgn', 'games/partita_0813-4'), ('tommidim_vs_roccot01_2022.05.29.pgn', 'games/partita_0529-2'), ('roccot01_vs_tommidim_2022.05.29 (2).pgn', 'games/partita_0529-5'), ('tommidim_vs_roccot01_2022.05.25 (1).pgn', 'games/partita_0525-1'), ('valesepicacchi_vs_gggarce_2022.08.13 (1).pgn', 'games/partita_0813-2'), ('tommidim_vs_roccot01_2022.05.29 (1).pgn', 'games/partita_0529-4'), ('roccot01_vs_tommidim_2022.05.29 (1).pgn', 'games/partita_0529-3'), ('valesepicacchi_vs_Sanjatosti_2022.08.14.pgn', 'games/partita_0814-1'), ('roccot01_vs_tommidim_2022.08.16.pgn', 'games/partita_0816-2'), ('tommidim_vs_roccot01_2022.08.16.pgn', 'games/partita_0816-5'), ('roccot01_vs_tommidim_2022.08.16.pgn', 'games/partita_0816-4'), ('roccot01_vs_tommidim_2022.05.29.pgn', 'games/partita_0529-1'), ('tommidim_vs_roccot01_2022.05.25.pgn', 'games/partita_0525-3'), ('roccot01_vs_tommidim_2022.05.25.pgn', 'games/partita_0525-2'), ('gggarce_vs_valesepicacchi_2022.08.13 (3).pgn', 'games/partita_0813-1')]
listozzo[1] 

('roccot01_vs_tommidim_2022.08.16.pgn', 'games/partita_0816-6')

In [8]:
# os.chdir("/Users/tommasodimario/Documents/GitHub/statistics-101/project/src")
# os.chdir(listozzo[11][1])
# filename = f"{listozzo[11][0]}"
# print(listozzo[11])

# file = open(f"{filename[:-4]}.csv", "w")
# pgn = read_pgn(filename)
# writer = csv.writer(file)

# columns = ["Move", "TimeUsed", "TotalTime", "Eval"]
# writer.writerow(columns)
# for row in pgn:
#     writer.writerow(row)

# ## Problemi con le partite numero: 9 pos referenced before assignment
# ##                                 11 list index out of range 
# ##                                 16 list index out of range
# ##                                 18 pos referenced before assignment 
# ##                                 19 pos referenced before assignment 

In [111]:
file = open("prova.csv", "w")

writer = csv.writer(file)

columns = ["Time", "Game", "Player", "Statistics", "Value"]
writer.writerow(columns)

#pseudo code
# to do create queste cose così da poter iterare in questa roba della morte e creare un dataset di questo tipo
# https://raw.githubusercontent.com/QuantEcon/lecture-python/master/source/_static/lecture_specific/pandas_panel/realwage.csv

print(os.getcwd())


lista_partite = []
gamecount = 0
count = 0   
for root,dirs, files in os.walk("games"):
    if str(root)  != "games":

        partita = f"Game{root[-6:]}"
        lista_partite.append(root[-6:])
        gamecount += 1
        
    row_written = 0
    flag = False
    row_1 = []
    row_2 = []
    row_3 = [] 
    row_4 = []
    row_5 = []
    row_6 = []
    for file in files:
        #qui siamo dentro i file di una singola partita
        filename = os.fsdecode(file)
        #print(filename)
        if filename.endswith(".txt") and (filename[0] == "w" or filename[0] == "b"): # abbiamo un carissimo file delle attività
        #leggiamo i battiti
            #print(f"processando {filename}")
            player = filename[0]
            activity = read_activity(f"{str(root)}/{filename}")
            # As explained in the markdown above on our strumentation TODO our smartwatches recorded the heartbeats at different time so we smoothed it out
            for hr, time in activity:
                row = [int(time), partita, player, "HR", int(hr)]
                count +=1
                
                if filename[0] == "w":
                    row_1.append(row)
                elif filename[0] == "b": 
                    row_2.append(row)
            row_written += 1
            
        elif filename.endswith(".pgn"):
            # #print(get_moves(f"{str(root)}/{filename}",), filename, partita)
            filename = filename[:-3] + "csv"
            #print(f"processando {filename}")
            
            try: 
                with open(f"{str(root)}/{filename}", newline='') as f:
                    pgn = []
                    for row in csv.reader(f):
                        if row[0] == "Move":
                            continue
                        
                        tupla = eval(row[2])
                        pos = row[1]
                        if "#" in row[1] or "Mate" in row[1]:
                            if "-" in pos:
                                pos = -20
                            else:
                                pos = +20
                            # #print(row[1], pos)
                        pgn.append([int(row[0]), float(pos), (float(tupla[0]),float(tupla[0])), eval(row[3]) ])
                        # #print([int(row[0]), float(pos), (float(tupla[0]),float(tupla[0])), eval(row[3]) ])
                pgn = pgn[1:]
                last_second = len(pgn) - 1
                
                    
                for second in range(len(pgn)):
                    ev = pgn[second][1]
                    try:
                        ev = float(pgn[second][1])
                    except:
                        ev = 20
                    row = [second, partita, "w", "Evaluation", ev]
                    
                    row_3.append(row)
                    row = [second, partita, "b", "Evaluation", -ev]
                    row_4.append(row)
                    
                
                for second in range(len(pgn)):
                    
                    row = [second, partita, "w", "TimeUsed", int(pgn[second][2][0])]
                    row_5.append(row)
                    row = [second, partita, "b", "TimeUsed", int(pgn[second][2][1])]
                    row_6.append(row)
                
                row_written += 1
            except:
                continue
                    
    if row_written == 3:
        
        list_rows = [row_1,row_2,row_3,row_4, row_5, row_6]
        length = min([len(row) for row in list_rows])
        
        
        
        count = 0
        for i in range(6):
            
            for r in range(length):
                
                writer.writerow([count] + list_rows[i][r])
                count += 1
                
            row_written = False
print(lista_partite)



/Users/tommasodimario/Documents/GitHub/statistics-101/project/src
925
864
869
1233
492
1036
531
1125
815
918
721
1136
572
1246
591
964
962
1117
842
['0814-2', '0816-6', '0816-1', '0816-7', '0813-3', '0813-4', '0529-2', '0529-5', '0813-2', '0529-4', '0529-3', '0814-1', '0816-2', '0816-5', '0816-4', '0816-3', '0529-1', '0813-1']


In [125]:

#pseudo code
# to do create queste cose così da poter iterare in questa roba della morte e creare un dataset di questo tipo
# https://raw.githubusercontent.com/QuantEcon/lecture-python/master/source/_static/lecture_specific/pandas_panel/realwage.csv





file = open("ex.csv", "w")

writer = csv.writer(file)
# TR: Time Remaining
# T: Time in seconds from the beginning of the match
# HRDelta: difference of the heart rate at the moment of the move
# TPR: Time per move
columns = ["Move", "Player", "TR", "T", "HRDelta", "TPR"]

writer.writerow(columns)
gamecount = 0
count = 0   
for root,dirs, files in os.walk("games"):
    if str(root)  != "games":
        partita = f"Game{root[-6:]}"
        gamecount += 1
        
    df = pd.read_csv("prova.csv")

    df = df.pivot_table(values='Value',
                            index='Time',
                            columns=['Game', 'Player', 'Statistics'])
    for file in files:
        #qui siamo dentro i file di una singola partita
        filename = os.fsdecode(file)
        print(filename)
        
        #leggiamo i battiti
        
        if filename.endswith(".pgn"):
            
            filename = filename[:-3] + "csv"
            print(f"processando {filename}")
            
        
            moves = get_moves(f"{str(root)}/{filename}")[:-1]
            print(moves)
            added = 0
            ret = []
            with open(f"{str(root)}/{filename}", newline='') as f:
                pgn = []
                time_last_move_w = 0
                time_last_move_b = 0
                for row in csv.reader(f):
                    if row[0] == "Move":
                        continue
                    
                    move = eval(row[3])
                    if move:
                        trem = eval(row[2])
                        
                        for _ in move:
                            if added % 2 == 0:
                                ret.append(moves[added] + [move,"w",trem[0],int(row[0]), df[partita]["w"]["HR"][int(row[0])], int(row[0]) - time_last_move_w]) 
                                time_last_move_w = int(row[0])
                            else:
                                ret.append(moves[added] + [move,"b",trem[1],int(row[0]), df[partita]["w"]["HR"][int(row[0])], int(row[0]) - time_last_move_b])
                                time_last_move_b = int(row[0])
                            added += 1
                        
for move in moves:
    writer.writerow(move)
                        
                        



.DS_Store
zSanjatosti_vs_valesepicacchi_2022.08.14 (1).pgn
processando zSanjatosti_vs_valesepicacchi_2022.08.14 (1).csv
[['e4'], ['e5'], ['f3'], ['Nf6'], ['d3'], ['d5'], ['Nc3'], ['dxe4'], ['dxe4'], ['Nc6'], ['Qxd8+'], ['Nxd8'], ['Bg5'], ['Be7'], ['Bxf6'], ['Bxf6'], ['O-O-O'], ['O-O'], ['Nge2'], ['c5'], ['Nd5'], ['Bg5+'], ['f4'], ['exf4'], ['Ndxf4'], ['Nc6'], ['h4'], ['Bxf4+'], ['Nxf4'], ['Rd8'], ['Nd5'], ['Bg4'], ['Rd2'], ['Re8'], ['Nc7'], ['Rad8'], ['Nxe8'], ['Rxe8'], ['Bc4'], ['Rxe4'], ['Bd5'], ['Rd4'], ['Bxc6'], ['bxc6'], ['Rxd4'], ['cxd4'], ['Re1'], ['h5'], ['Re7'], ['a5'], ['Rc7'], ['Bf5'], ['Rxc6'], ['d3'], ['cxd3'], ['Bxd3'], ['Rb6'], ['f6'], ['b4'], ['axb4'], ['Rxb4'], ['g5'], ['hxg5'], ['fxg5'], ['Rb5'], ['Bxb5'], ['a3'], ['g4'], ['g3'], ['h4'], ['gxh4'], ['g3'], ['Kd2'], ['g2'], ['Ke3'], ['g1=Q+'], ['Kf3'], ['Qh1+'], ['Kg3'], ['Qa1'], ['h5'], ['Qxa3+'], ['Kg4'], ['Kh7'], ['Kg5'], ['Qc5+'], ['Kh4'], ['Be2']]
[['e4', ['e2e4'], 'w', 0.0, 0, -1.0, 0], ['e5', ['e7e5'], 'b', 0.1, 

KeyError: 1189

In [97]:
#Adesso che abbiamo i dettagli delle mosse ci ripeschiamo dal dataset madre il battito al tempo della mossa
realwage = pd.read_csv("prova.csv")

realwage = realwage.pivot_table(values='Value',
                                index='Time',
                                columns=['Game', 'Player', 'Statistics'])

## ora dobbiamo aggiungere a ogni mossa



-7.0

NameError: name 'list_rows' is not defined

In [ ]:

# Display 6 columns for viewing purposes
# pd.set_option('display.max_columns', 6)

# Reduce decimal points to 2
# pd.options.display.float_format = '{:,.2f}'.format

realwage = pd.read_csv("prova.csv")
realwage = realwage.pivot_table(values='Value',
                                index='Time',
                                columns=['Game', 'Player', 'Statistics'])
realwage

Game            Game0                                          
Player              b                          w               
Statistics Evaluation     HR TimeUsed Evaluation    HR TimeUsed
Time                                                           
0               -0.38  107.0      0.0       0.38  88.0      0.0
1               -0.55  107.0      0.0       0.55  88.0      0.0
2               -0.55  107.0      0.0       0.55  89.0      0.0
3               -0.55  107.0      0.0       0.55  89.0      0.0
4               -0.55  107.0      0.0       0.55  90.0      0.0
...               ...    ...      ...        ...   ...      ...
864            -20.00  101.0      NaN      20.00  78.0      NaN
865            -20.00  102.0      NaN      20.00  78.0      NaN
866            -20.00  102.0      NaN      20.00  78.0      NaN
867            -20.00  103.0      NaN      20.00  78.0      NaN
868            -20.00  103.0      NaN      20.00  78.0      NaN

[869 rows x 6 columns]

In [ ]:
realwage["Game0"]

Player              b                          w               
Statistics Evaluation     HR TimeUsed Evaluation    HR TimeUsed
Time                                                           
0               -0.38  107.0      0.0       0.38  88.0      0.0
1               -0.55  107.0      0.0       0.55  88.0      0.0
2               -0.55  107.0      0.0       0.55  89.0      0.0
3               -0.55  107.0      0.0       0.55  89.0      0.0
4               -0.55  107.0      0.0       0.55  90.0      0.0
...               ...    ...      ...        ...   ...      ...
864            -20.00  101.0      NaN      20.00  78.0      NaN
865            -20.00  102.0      NaN      20.00  78.0      NaN
866            -20.00  102.0      NaN      20.00  78.0      NaN
867            -20.00  103.0      NaN      20.00  78.0      NaN
868            -20.00  103.0      NaN      20.00  78.0      NaN

[869 rows x 6 columns]